In [14]:
import warnings
warnings.filterwarnings('ignore')

import os
import torch
import pinecone
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModelForCausalLM, pipeline
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_ollama import ChatOllama

In [15]:
import textwrap

def view_text(texts):
    paragraphs = texts.split('\n')
    for paragraph in paragraphs:
        lines = textwrap.wrap(paragraph, width=100)
        for line in lines:
            print(line)
    print("\n")

# Load  embeddings + Model

In [7]:
# Load environment variables
load_dotenv(r"D:\Online_Learning\Practical_DL\langchain_udemy\ice_breaker\.env")

# Retrieve the API key and index name from environment variables
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
INDEX_NAME = os.getenv("INDEX_NAME")

pc = Pinecone(api_key=PINECONE_API_KEY)

llm = ChatOllama(model="llama3.2:3b", use_gpu=True, max_tokens=1024, temperature=0.7, top_p=0.9)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Initialize Pinecone vector store
docsearch = PineconeVectorStore(index_name=INDEX_NAME, embedding=embeddings)

# Initialize conversation memory with summary
memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history", return_messages=True)


# Chat Template

In [31]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

# Create a chat template with system and human messages
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content='You respond only in the JSON format.'),
        HumanMessagePromptTemplate.from_template('Top {n} countries in {area} by population.')
    ]
)

# Fill in the specific values for n and area
messages = chat_template.format_messages(n='5', area='World')
print(messages)  # Outputs the formatted chat messages

output = llm.invoke(messages)
print(output.content)


[SystemMessage(content='You respond only in the JSON format.'), HumanMessage(content='Top 5 countries in World by population.')]
{
    "countries": [
        {
            "rank": 1,
            "country": "China",
            "population": 1439323776
        },
        {
            "rank": 2,
            "country": "India",
            "population": 1380004385
        },
        {
            "rank": 3,
            "country": "United States",
            "population": 331449281
        },
        {
            "rank": 4,
            "country": "Indonesia",
            "population": 273523615
        },
        {
            "rank": 5,
            "country": "Pakistan",
            "population": 216565317
        }
    ]
}


# Sequential Chain

In [30]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain


# Define the first prompt template
prompt_template1 = PromptTemplate.from_template(
    template='You are an experienced scientist and Python programmer. Write a function that implements the concept of {concept}.'
)
# Create an LLMChain using the first model and the prompt template
chain1 = LLMChain(llm=llm, prompt=prompt_template1)

# Define the second prompt template
prompt_template2 = PromptTemplate.from_template(
    template='Given the Python function {function}, describe it as detailed as possible.'
)
# Create another LLMChain using the second model and the prompt template
chain2 = LLMChain(llm=llm, prompt=prompt_template2)

# Combine both chains into a SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)

# Invoke the overall chain with the concept "linear regression"
output = overall_chain.invoke('linear regression')


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_27856\3164632939.py:11: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  chain1 = LLMChain(llm=llm, prompt=prompt_template1)




> Entering new SimpleSequentialChain chain...
import numpy as np
from scipy import stats

def linear_regression(x, y):
    """
    This function performs simple linear regression.

    Parameters:
    x (list or array): Independent variable.
    y (list or array): Dependent variable.

    Returns:
    slope (float): The slope of the best fit line.
    intercept (float): The intercept of the best fit line.
    """

    # Convert lists to numpy arrays for easier calculations
    x = np.array(x)
    y = np.array(y)

    # Calculate the mean of x and y
    mean_x = np.mean(x)
    mean_y = np.mean(y)

    # Calculate the deviations from the mean for x and y
    dev_x = x - mean_x
    dev_y = y - mean_y

    # Calculate the slope using the formula: m = Σ[(xi-x̄)(yi-ȳ)] / Σ(xi-x̄)^2
    numerator = np.sum(dev_x * dev_y)
    denominator = np.sum(dev_x ** 2)
    slope = numerator / denominator if denominator != 0 else 0

    # Calculate the intercept using the formula: b = ȳ - m*x̄
    interc

In [32]:
# Define the first prompt template
prompt_template1 = PromptTemplate.from_template(
    template='You are an experienced scientist in NLP and Deep Learning Engineer. Write a function in Python that implements the concept of {concept}.'
)
# Create an LLMChain using the first model and the prompt template
chain1 = LLMChain(llm=llm, prompt=prompt_template1)

# Define the second prompt template
prompt_template2 = PromptTemplate.from_template(
    template='Given the Python function {function}, describe it as detailed as possible.'
)
# Create another LLMChain using the second model and the prompt template
chain2 = LLMChain(llm=llm, prompt=prompt_template2)

# Combine both chains into a SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)

# Invoke the overall chain with the concept "linear regression"
output = overall_chain.invoke('syntactic parser')




> Entering new SimpleSequentialChain chain...
# Syntactic Parser Function

In this example, we will use a simple recursive descent parser to parse a syntax tree for a basic grammar.

```python
import re

class Node:
    """Represents a node in the syntax tree."""
    
    def __init__(self, value):
        self.value = value
        
    def __str__(self):
        return f"Node({self.value})"


class Parser:
    """A simple syntactic parser."""
    
    def __init__(self, grammar):
        self.grammar = grammar
        self.tokenizer = Tokenizer()
        
    def parse(self, sentence):
        """Parses a sentence using the provided grammar."""
        
        # Split the sentence into tokens
        tokens = self.tokenizer.tokenize(sentence)
        
        # Start parsing at the first token
        return self.parse_token(tokens[0])
    
    def parse_token(self, token):
        """Parses a single token."""
        
        # Check if the token matches any of the grammar rules


# Creating React Agent

In [27]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import Tool, AgentExecutor, initialize_agent, create_react_agent
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama

# Initialize the ChatOpenAI model (gpt-4-turbo-preview) with a temperature of 0. Utilize gpt-3.5-turbo if you use the free plan
# llm = ChatOpenAI(model_name='gpt-4-turbo-preview', temperature=0)

llm = ChatOllama(model="llama3.2:3b", use_gpu=True, max_tokens=2048, temperature=0.5, top_p=0.95)

# Define a template for answering questions
template = '''
Answer the following questions in English as best you can.
Questions: {q}
'''

# Create a PromptTemplate object from the template
prompt_template = PromptTemplate.from_template(template)

# Pull the react prompt from the hub
prompt = hub.pull('hwchase17/react')

# displaying information about the react prompt
# print(type(prompt))
# print(prompt.input_variables)
# print(prompt.template)


# Create tools for the agent

# 1. Python REPL Tool (for executing Python code)
python_repl = PythonREPLTool()
python_repl_tool = Tool(
    name='Python REPL',
    func=python_repl.run,
    description='Useful when you need to use Python to answer a question. You should input Python code.'
)

# 2. Wikipedia Tool (for searching Wikipedia)
api_wrapper = WikipediaAPIWrapper()
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper)
wikipedia_tool = Tool(
    name='Wikipedia',
    func=wikipedia.run,
    description='Useful for when you need to look up a topic, country, or person on Wikipedia.'
)

# 3. DuckDuckGo Search Tool (for general web searches)
search = DuckDuckGoSearchRun()
duckduckgo_tool = Tool(
    name='DuckDuckGo Search',
    func=search.run,
    description='Useful for when you need to perform an internet search to find information that another tool can\'t provide.'
)

# Combine the tools into a list
tools = [python_repl_tool, wikipedia_tool, duckduckgo_tool]

# Create a react agent with the ChatOpenAI model, tools, and prompt
agent = create_react_agent(llm, tools, prompt)

# Initialize the AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=10
)


d:\Programs\Anaconda\envs\pytorch\lib\site-packages\langsmith\client.py:351: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [28]:
# question = 'Generate the first 20 numbers in the Fibonacci series.'
# question = 'Who is the current prime minister of the UK?'

question = 'Tell me about Napoleon Bonaparte early life in 4 sentences'
output = agent_executor.invoke({
    'input': prompt_template.format(q=question)
})




> Entering new AgentExecutor chain...
Action: Wikipedia
Action Input: 'Napoleon Bonaparte early lifePage: Lucien Bonaparte
Summary: Lucien Bonaparte, 1st Prince of Canino and Musignano (born Luciano Buonaparte; 21 May 1775 – 29 June 1840), was a French politician and diplomat of the French Revolution and the Consulate. He served as Minister of the Interior from 1799 to 1800 and as the president of the Council of Five Hundred in 1799.
The third surviving son of Carlo Bonaparte and his wife Letizia Ramolino, Lucien was the younger brother of Napoleon Bonaparte. As president of the Council of Five Hundred, he was one of the participants of the Coup of 18 Brumaire that brought Napoleon to power in France.

Page: Napoleon III
Summary: Napoleon III (Charles-Louis Napoléon Bonaparte; 20 April 1808 – 9 January 1873) was the first president of France from 1848 to 1852, and the last monarch of France as the second Emperor of the French from 1852 until he was deposed on 4 September 1870.
Prior 

In [29]:
print(output['input'])
print(output['output'])


Answer the following questions in English as best you can.
Questions: Tell me about Napoleon Bonaparte early life in 4 sentences

Agent stopped due to iteration limit or time limit.


# Saving chat sessions

In [13]:
import textwrap
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import SystemMessage
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder


llm = ChatOllama(model="llama3.2:3b", use_gpu=True, max_tokens=1024, temperature=0.7, top_p=0.9)

def view_text(texts):
    paragraphs = texts.split('\n')
    for paragraph in paragraphs:
        lines = textwrap.wrap(paragraph, width=100)
        for line in lines:
            print(line)
    print("\n")


# 2. Create memory 
memory = ConversationBufferMemory(
    memory_key='chat_history',
    return_messages=True
)

# 3. add  MessagesPlaceholder(variable_name='messages') to the prompt
prompt = ChatPromptTemplate(
    input_variables=["content", "chat_history"],
    messages=[
        SystemMessage(content="You are a chatbot having a conversation with a human."),
        MessagesPlaceholder(variable_name="chat_history"), # Where the memory will be stored.
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

# 4. Add the memory to the chain
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False
)

while True:
    content = input('Your prompt: ')
    if content.lower() in ['quit', 'exit', 'bye']:
        print('Goodbye!')
        break
    
    response = chain.invoke({'content': content})
    print(response['content'])
    # print(response['chat_history'])
    print('')
    print(view_text(response['text']))
    print('-' * 50)
    print('')

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_22720\2460789804.py:36: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  chain = LLMChain(


Goodbye!


# Saving chat sections in RAG

In [29]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

system_template = '''
Use the following pieces of context to answer the user's question.
If you don't find the answer in the provided context, just respond "I don't know."
---------------
Context: ```{context}```
'''

user_template = '''
Question: ```{question}```
'''

messages= [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template(user_template)
]

qa_prompt = ChatPromptTemplate.from_messages(messages)

# Create a conversational retrieval chain with memory
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=docsearch.as_retriever(),
    memory=memory,
    chain_type='stuff',  # Specify the chain type
    combine_docs_chain_kwargs={'prompt': qa_prompt},
    verbose=False  # Set to True to enable verbose logging for debugging
)

def ask_question(q, chain):
    result = chain.invoke({'question': q})
    return result

print("Chatbot: Hello! How can I assist you today?")
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit', 'bye']:
        print("Chatbot: Goodbye!")
        break
    else:
        # Use the conversational retrieval chain to get the response
        response = ask_question(user_input, qa_chain)
        
        # Retrieve and print the chat history
        print("Chatbot: ", user_input)
        view_text(response['answer'])


Chatbot: Hello! How can I assist you today?
Chatbot:  What is Corpus Reader?
A Corpus Reader is a utility object in NLTK (Natural Language Toolkit) that provides a way to
quickly create corpora and associate them with readers. It is a generic reader that can be used to
read various types of corpus formats, such as plain-text documents, part-of-speech tagged corpora,
and chunked corpora, among others.


Chatbot:  What is NLTK?
A Corpus Reader in NLTK is an object that can access and read corpora (collections of texts). The
capabilities of a Corpus Reader include:
* Providing information about the corpus structure
* Allowing you to create corpora from plain text data
* Associating readers with corpora
* Giving hints on how to customize a CorpusReader for application-specific purposes
Some examples of utility readers provided by NLTK include:
- PlaintextCorpusReader (for plain-text documents)
- TaggedCorpusReader (for simple part-of-speech tagged corpora)
- BracketParseCorpusReader (for p

# Test with ChromaDB

In [21]:
from langchain.vectorstores import Chroma

# loading PDF, DOCX and TXT files as LangChain Documents
def load_document(file):
    import os
    _, extension = os.path.splitext(file)

    if extension == '.pdf':
        from langchain_community.document_loaders import PyPDFLoader
        print(f'Loading {file}')
        loader = PyPDFLoader(file)
    elif extension == '.docx':
        from langchain_community.document_loaders import Docx2txtLoader
        print(f'Loading {file}')
        loader = Docx2txtLoader(file)
    elif extension == '.txt':
        from langchain_community.document_loaders import TextLoader
        loader = TextLoader(file)
    else:
        print('Document format is not supported!')
        return None

    data = loader.load()
    return data

# splitting data in chunks
def chunk_data(data, chunk_size=256, chunk_overlap=20):
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = text_splitter.split_documents(data)
    return chunks


# create embeddings using OpenAIEmbeddings() and save them in a Chroma vector store
def create_embeddings(chunks):
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")  # 512 works as well
    # vector_store = Chroma.from_documents(chunks, embeddings)

    # if you want to use a specific directory for chromadb
    vector_store = Chroma.from_documents(chunks, embeddings, persist_directory='./application_with_langchain/mychroma_db_test')
    return vector_store

def calculate_embedding_cost(texts):
    import tiktoken
    enc = tiktoken.encoding_for_model('text-embedding-3-small')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    # check prices here: https://openai.com/pricing
    # print(f'Total Tokens: {total_tokens}')
    # print(f'Embedding Cost in USD: {total_tokens / 1000 * 0.00002:.6f}')
    return total_tokens, total_tokens / 1000 * 0.00002


In [56]:
data = load_document('D:/Online_Learning/Practical_DL/doc/1910.13461v1.pdf')
chunks = chunk_data(data, chunk_size=1000)
tokens, embedding_cost = calculate_embedding_cost(chunks)

Loading D:/Online_Learning/Practical_DL/doc/1910.13461v1.pdf


In [57]:
vector_store = create_embeddings(chunks)

In [62]:
q = 'Ask a question about the content of your file: What is MNLI? Search in the DB'

def ask_and_get_answer(vector_store, q, k=3):
    from langchain.chains import RetrievalQA

    llm = ChatOllama(model="llama3.2:3b", use_gpu=True, max_tokens=2048, temperature=0.5, top_p=0.95)
    retriever = vector_store.as_retriever()
    chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

    answer = chain.invoke(q)
    return answer['result']

answer = ask_and_get_answer(vector_store, q, k=3)

# Test Creating Index with Pinecone

In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import pinecone
import streamlit as st
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_ollama import ChatOllama

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModelForCausalLM, pipeline
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain.memory import ConversationSummaryMemory
from langchain.chains import RetrievalQA
from langchain_ollama import ChatOllama

In [3]:
def load_document(file):
    import os
    name, extension = os.path.splitext(file)

    if extension == '.pdf':
        from langchain.document_loaders import PyPDFLoader
        print(f'Loading {file}')
        loader = PyPDFLoader(file)
    elif extension == '.docx':
        from langchain.document_loaders import Docx2txtLoader
        print(f'Loading {file}')
        loader = Docx2txtLoader(file)
    elif extension == '.txt':
        from langchain.document_loaders import TextLoader
        loader = TextLoader(file)
    else:
        print('Document format is not supported!')
        return None

    data = loader.load()
    return data


# splitting data in chunks
def chunk_data(data, chunk_size=1000, chunk_overlap=20):
    from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30, separator="\n")
    chunks = text_splitter.split_documents(data)
    return chunks


# Insert to Pinecone index
def insert_or_fetch_embeddings(index_name, chunks):
    # importing the necessary libraries and initializing the Pinecone client
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    # loading from existing index
    if (index_name in pc.list_indexes().names()):
        print(f'Index {index_name} already exists. Loading embeddings ... ', end='')
        print('Ok')
    else:
        # creating the index and embedding the chunks into the index 
        print(f'Creating index {index_name} and embeddings ...', end='')

        # creating a new index
        pc.create_index(
            name=index_name,
            dimension=384,
            metric='cosine',
            spec=ServerlessSpec(
                cloud="aws",
                region="us-east-1"
        ) 
        )

        PineconeVectorStore.from_documents(chunks, embeddings, index_name=index_name)

        # processing the input documents, generating embeddings using the provided `OpenAIEmbeddings` instance,
        # inserting the embeddings into the index and returning a new Pinecone vector store object. 
        
        print('Ok')

    vector_store = PineconeVectorStore(index_name=index_name, embedding=embeddings)

    return vector_store


def delete_pinecone_index(index_name='all', exclude_indexes=['documents-embeddings-index']):
    if index_name == 'all':
        indexes = [i for i in pc.list_indexes().names() if i not in exclude_indexes]
        print('Deleting all indexes ... ')
        for index in indexes:
            print(index)
            pc.delete_index(index)
        print('Ok')
    else:
        print(f'Deleting index {index_name} ...', end='')
        pc.delete_index(index_name)
        print('Ok')

    
def ask_and_get_answer(vector_store, q, k=5):
    llm = ChatOllama(model="llama3.2:3b", use_gpu=True, max_tokens=2048, temperature=0.5, top_p=0.95)
    retriever = vector_store.as_retriever(search_kwargs={'k': k})
    chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

    answer = chain.invoke(q)
    return answer['result']


# calculate embedding cost using tiktoken
def calculate_embedding_cost(texts):
    import tiktoken
    enc = tiktoken.encoding_for_model('text-embedding-3-small')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    # check prices here: https://openai.com/pricing
    # print(f'Total Tokens: {total_tokens}')
    # print(f'Embedding Cost in USD: {total_tokens / 1000 * 0.00002:.6f}')
    return total_tokens, total_tokens / 1000 * 0.00002


# clear the chat history from streamlit session state
def clear_history():
    if 'history' in st.session_state:
        del st.session_state['history']

In [6]:
# Load environment variables
load_dotenv()

# Retrieve the API key and index name from environment variables
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
INDEX_NAME = os.getenv("INDEX_NAME")

pc = Pinecone(api_key=PINECONE_API_KEY)

In [75]:
file_path = 'D:/Online_Learning/Practical_DL/doc/1910.13461v1.pdf'
data = load_document(file_path)
chunks = chunk_data(data, chunk_size=1000)
tokens, embedding_cost = calculate_embedding_cost(chunks)
print("Total Tokens:", tokens)
print("Total cost (USD):", embedding_cost)

Loading D:/Online_Learning/Practical_DL/doc/1910.13461v1.pdf
Total Tokens: 11139
Total cost (USD): 0.00022278


In [78]:
index_name = 'test-file-1'
vector_store = insert_or_fetch_embeddings(index_name=index_name, chunks=chunks)

print("Finish chunking and embedding the file.")
print(vector_store)

Creating index test-file-1 and embeddings ...Ok
Finish chunking and embedding the file.


In [81]:
q = 'Using provided data. What is MNLI?'
answer = ask_and_get_answer(vector_store, q, k=3)
view_text(answer)

According to the provided context, MNLI stands for Multi-Genre Natural Language Inference (NLI).
It's a bitext classification task where the goal is to predict whether one sentence entails another.
The model fine-tunes by concatenating two sentences with an appended EOS token and passing them
through both the BART encoder and decoder.




In [83]:
def delete_pinecone_index(index_name='all', exclude_indexes=['documents-embeddings-index']):
    if index_name == 'all':
        indexes = [i for i in pc.list_indexes().names() if i not in exclude_indexes]
        print('Deleting all indexes ... ')
        for index in indexes:
            print(index)
            pc.delete_index(index)
        print('Ok')
    else:
        print(f'Deleting index {index_name} ...', end='')
        pc.delete_index(index_name)
        print('Ok')

delete_pinecone_index(index_name='all', exclude_indexes=['documents-embeddings-index'])

Deleting all indexes ... 
test-file-1
Ok


In [4]:
data = load_document('D:/Online_Learning/Practical_DL/doc/1910.13461v1.pdf')
chunks = chunk_data(data, chunk_size=1000)
tokens, embedding_cost = calculate_embedding_cost(chunks)
index_name = 'uae-test2-index'

Loading D:/Online_Learning/Practical_DL/doc/1910.13461v1.pdf


In [7]:
# Insert to Pinecone index
def insert_or_fetch_embeddings(index_name, chunks):
    # Initialize the model
    embeddings = HuggingFaceEmbeddings(model_name="WhereIsAI/UAE-Large-V1")

    # Check if the index already exists
    if index_name in pc.list_indexes().names():
        print(f'Index {index_name} already exists. Loading embeddings ... ', end='')
        print('Ok')
    else:
        # Create the index and embed the chunks into the index
        print(f'Creating index {index_name} and embeddings ...', end='')

        # Create a new index
        pc.create_index(
            name=index_name,
            dimension=1024,
            metric='cosine',
            spec=ServerlessSpec(
                cloud="aws",
                region="us-east-1"
            )
        )

        PineconeVectorStore.from_documents(chunks, embeddings, index_name=index_name)

        print('Ok')

    # Initialize the Pinecone vector store
    vector_store = PineconeVectorStore(index_name=index_name, embedding=embeddings)

    return vector_store

vector_store = insert_or_fetch_embeddings(index_name, chunks)

Creating index uae-test2-index and embeddings ...Ok


In [ ]:
vector_store

NameError: name 'vector_store' is not defined

: 